In [77]:
import pandas as pd
import spacy
import requests
import datetime
import os
from pathlib import Path
from dotenv import load_dotenv
import openai
import anthropic

In [78]:

load_dotenv()

OPENAI_API_KEY      = os.getenv('OPENAI_API_KEY')
PERPLEXITY_API_KEY  = os.getenv('PERPLEXITY_API_KEY')
ANTHROPIC_API_KEY   = os.getenv('ANTHROPIC_API_KEY')
TEST                = os.getenv('TEST')
HUMAN_FILE          = os.getenv('HUMAN_FILE')

print(TEST, HUMAN_FILE)

0 imports/HUMAN-TEXTS-1


In [79]:
oai_client = openai.OpenAI()
ant_client = anthropic.Anthropic()



In [80]:
def export_full_text(tl, filename, marker='\n--------------------NEW-TEXT------------------\n'):
    with open(filename, 'w') as f:
        f.write(marker.join(tl))

In [81]:
def gpt4o_request(prompt, system_prompt):
    completion = oai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "text"
        },
        temperature=1,
        max_tokens=1993,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    return str(completion.choices[0].message.content)

def gpt35t_request(prompt, system_prompt):
    completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "text"
        },
        temperature=1,
        max_tokens=1993,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    return str(completion.choices[0].message.content)

def perplxty_request(prompt, system_prompt):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json",
    }
    data = {
        "model": "llama-3.1-sonar-large-128k-online",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1993,
        "temperature": 0.5,
        "top_p": 1
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"].strip()

def clde_request(prompt, system_prompt):
    message = ant_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1993,
        temperature=0.5,
        system=system_prompt,  # Top-level system parameter
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
    )
    return message.content[0].text


In [82]:
prompt          = []
system_prompt   = 'Schreiben Sie einen Zeitungsartikel, der dem Sport zuzuordnen ist und alle Anforderungen an fachliche und inhaltliche Korrektheit erfüllt.\n\n- Achten Sie darauf, sachliche Informationen zu vermitteln, die den aktuellen Stand des Sportereignisses oder Themas widerspiegeln.\n- Verwenden Sie eine klare und ansprechende Sprache, die das Interesse der Leser weckt.\n- Sorgen Sie für eine ausgewogene Berichterstattung, indem Sie verschiedene Perspektiven einbeziehen, falls relevant.\n\n# Schritte\n\n1. Sammeln Sie relevante Informationen und Daten, um eine fundierte Berichterstattung sicherzustellen.\n2. Schreiben Sie eine einleitende Passage, die das Thema und seine Wichtigkeit kurz beschreibt.\n3. Gliedern Sie den Hauptteil des Artikels in sinnvolle Abschnitte, die unterschiedliche Aspekte des Themas beleuchten, wie Hintergrundinformationen, aktuelle Entwicklungen oder Prognosen.\n4. Fügen Sie relevante Zitate und Meinungen von Experten oder Beteiligten ein, um den Artikel zu bereichern.\n5. Schreiben Sie einen Schluss, der die Hauptpunkte des Artikels zusammenfasst und einen Ausblick oder abschließenden Gedanken bietet.\n6. Fassen Sie kurz die Spielergebnisse zusammen.\n7. Überprüfe, ob der Text der längenangabe entspricht! Falls nicht, verlängere den Text entsprechend\n\n# Ausgabeformat\n\n- Länge: Der Artikel muss mindestens 800 und maximal 900 Wörter umfassen!\n\n# Beispiel 1\n\nObschon der lange torlose, aber interessante WM-Final zunächst nicht für die Deutschen zu laufen scheint, behalten sie das bessere Ende für sich.\n\nDie Szenen nach der 120. Minute sprachen Bände: Javier Mascherano, der argentinische Kämpfer, lief mit letzter Kraft los, spielte Lionel Messi an, der antrat und von Bastian Schweinsteiger mit letztem Einsatz angegangen und gefoult wurde. Das war das 36. und letzte Foul in diesem intensiven Spiel mit vielen, hart geführten Zweikämpfen. Derweil sich Messi den Ball zum Freistoss hinlegte, blieb Schweinsteiger liegen. Er winkte ab, konnte nicht mehr mittun. Und Messi? Er hatte noch Luft, schoss aber weit über das Tor. Das wars. \n \n ## Frühe Wechsel\nVielleicht wäre alles anders gekommen, ja, vielleicht, wenn Messi nach der Pause die Torchance zum 1:0 genutzt hätte. Er fuhr sich stattdessen abermals ungläubig durchs Haar – wie nach dem Freistoss ganz zuletzt. Vielleicht hätte das Spiel einen anderen Verlauf genommen, wenn die Südamerikaner ihre Vorteile in der ersten Spielhälfte genutzt hätten. Ja, vielleicht. Viel sprach zunächst nicht für die im Vorfeld favorisierten Deutschen. Aber die Geschichte lehrt, dass sie genau in solchen Momenten Stärken ausspielen und vor allem nicht zu besiegen sind. Es war in der ersten halben Stunde nicht ihr Spiel, sie fanden vor dem gegnerischen Tor die Wege nicht, sie wehrten sich mit Fouls, wurden verwarnt (Schweinsteiger, Höwedes), mussten kurzfristig auf den angeschlagenen Sami Khedira verzichten, sahen sich wegen der Verletzung des Khedira-Ersatzes Christoph Kramer früh zum ersten Wechsel gezwungen.\n\n## Unterhaltsamer Final\nDas alles führte in der Summe dazu, dass die Not jener Auswahl, die das Turnier mit hohen Siegen gegen Portugal (4:0) und Brasilien (7:1) geprägt hatte, zunächst überraschend gross war – so gross, dass die Führung der Argentinier nur eine Frage der Zeit schien.\n\nGonzalo Higuaín konnte in der 21. Minute sogar allein auf das Tor zulaufen, nachdem ihm Toni Kroos den Ball per Kopf nach einer missglückten Rückgabe vorgelegt hatte. Doch Higuaín verzog den Schuss und jubelte etwas später zu früh, weil ein schneller Angriff im Offside abgeschlossen worden war. Auch Messi war zu sehen, als er Mats Hummels davonlief, Gefahr heraufbeschwor und die erste These des Abends bestätigte: Argentinien ist gut, Deutschland weniger. Wären die Minuten der Deutschen vor der Pause nicht gewesen, die Torchance des eingewechselten André Schürrle und vor allem der Pfostenkopfball Benedikt Höwedes, man hätte sich wirklich ernsthaft um die deutschen Spieler sorgen müssen.\n\nDer WM-Final 2014 war vor allem in der ersten Halbzeit unterhaltsamer als manche andere, auch die Verlängerung hatte es in sich: Zuerst prüfte Schürrle den argentinischen Goalie Sergio Romero, und in der 97. Minute hatte der eingewechselte Rodrigo Palacio sogar einen Matchball auf dem Fuss, aber sein Heber, allein vor dem Tor stehend, flog knapp vorbei. Was dem Final fehlte, waren einzig Tore, die aufgrund der Konstellation grössere Emotionen hätten zur Folge haben können.\n\n## Ähnlich wie 2010\nDoch zu diesem Endspiel passte, dass es nicht im Penaltyschiessen entschieden wurde – wie jenes 2010 in Südafrika, als der Spanier Andres Iniesta gegen Holland in der 116. Minute traf.\n\nJetzt, am Sonntag im Maracanã-Stadion, war es die 113. Minute, die die Entscheidung brachte. Pass Schürrles, Ballannahme und Schuss Mario Götzes – 1:0 für Deutschland. Nicht diejenigen gewannen, die im Final am Anfang überraschten und lange Zeit Vorteile hatten, nicht Messi, nicht Higuaín, nicht Palacio trotz grossen Torchancen. Nein. Deutschland schaffte es. Wieder einmal. Das passte nur halbwegs zur Physiognomie des Finals. Dafür aber umso mehr zum ganzen Turnier.\n\nDeutschland - Argentinien 1:0 n.V. (0:0, 0:0)\n\nMaracanã , Rio de Janeiro. - 74,738 Zuschauer. - Schiedsrichter: Rizzoli (It). - Tor: 113. Götze 1:0. \n \n Deutschland: Neuer; Lahm, Boateng, Hummels, Höwedes; Kramer (31. Schürrle), Schweinsteiger; Müller, Kroos, Özil (120. Mertesacker); Klose (88. Götze).\n\nArgentinien: Romero; Zabaleta, Demichelis, Garay, Rojo; Perez (86. Gago), Biglia, Mascherano, Lavezzi (46. Agüero); Messi, Higuain (78. Palacio).\n\nBemerkungen: Deutschland ohne Khedira und Mustafi (beide verletzt). Argentinien ohne Di Maria (verletzt). 30. Tor von Higuain wegen Abseits aberkannt. 31. Kramer angeschlagen ausgewechselt. 45. Kopfball von Höwedes an den Pfosten. Verwarnungen: 29. Schweinsteiger. 34. Höwedes. 64. Mascherano. 65. Agüero (alle Foul).\n\n\n# Beispiel 2\n\nMit einem mühevollen Sieg in der Verlängerung sichert die Mannschaft von Bundestrainer Joachim Löw Deutschland gegen Argentinien den vierten Weltmeistertitel nach 1954, 1974 und 1990.\n\nRio de Janeiro - Diese Weltmeisterschaft hat mit wundersamen Ereignissen nicht gespart. Da fügte sich das Finale gestern trefflich ein in den Gang der Dinge. Als Deutschland Weltmeister wurde und sich in Rio de Janeiro den vierten Stern, für jeden Titel einen auf dem Trikot, sicherte, kehrte Mario Götze vom FC Bayern München in die öffentliche Beachtung zurück.\n\nEigentlich zählte er zu den Verlieren dieser WM, er hatte sich aus der Mannschaft gespielt und eine enttäuschende WM zuletzt eher von der Bank erlebt. Aber als die Partie gegen Argentinien am Sonntag 113 Minuten lang hin und her gewogt war, spielte sich der kleine Götze als Einwechselspieler ins Rampenlicht zurück und noch weiter: Sein Siegtreffer in der Verlängerung beim 1:0-Erfolg gegen Argentinien sicherte den Titel vor 74 000 Zuschauern im überbuchten Maracanã-Stadion.\n\nNun fliegt die Mannschaft von Bundestrainer Joachim Löw am Montagabend als neuer Fußball-Weltmeister nach Berlin, wo sie sich am Dienstag auf der Fanmeile präsentieren wird. Zehntausende wie Heuschrecken in die Stadt eingefallene Argentinier verabschiedeten sich betrübt in die Nacht, derweil die klare Minderheit deutscher Anhänger eine unvergessene Nacht in Rio erlebte. Zu den Emotionen gesellt sich eine neue Faktenlage: Zumindest für Sporthistoriker und Fußballwissenschaftler ist es nämlich nicht unbedeutend, dass somit erstmals eine europäische Mannschaft auf dem amerikanischen Kontinent eine Fußball-Weltmeisterschaft gewann.\n\nFür den Trainer Löw und Spieler wie Philipp Lahm, Bastian Schweinsteiger oder Miroslav Klose ist dieser Triumph auch eine späte Vollendung ihres gemeinsamen fußballerischen Lebenswerks, das 2004 unter Jürgen Klinsmann entwickelt und von Löw fortgeführt wurde. Für den deutschen Fußball ist es gleichzeitig ebenfalls eine Vollendung: die des Talentfördersystems, das 2001 nach dem Rumpelfußball der EM 2000 installiert wurde und mittlerweile Verband und Vereinen mehr als hundert Millionen Euro pro Jahr wert ist.\n\nDabei schienen die Voraussetzungen eher weniger Optimismus stiftend: Kurz vor dem Anpfiff machte die Kunde die Runde, dass Sami Khedira passen musste − jener große Spielstratege, der von Löw häufig so gelobt wurde wegen seiner Präsenz, als gäbe er dessen verlängerten Arm auf den Feld. Für ihn schickte Löw den jungen Christoph Kramer zu seinem Startelfdebüt ausgerechnet im Finale auf den Platz. Obwohl Kramer ordentlich spielte, wirkte die Mannschaft prompt nicht mehr so pass- und kombinationssicher wie in den Spielen zuvor. Auch schien mit Khedira der Mut gewichen zu sein, dieses unbedingte Vertrauen in die eigene Favoritenrolle, das sich die deutsche Mannschaft durch den Kantersieg über Brasilien angeeignet zu haben schien.\n\nNeuer will ja nur spielen\nUnd in der ersten Spielhälfte zeigten die Argentinier die besseren Tormöglichkeiten. Lionel Messi gewann ein ums andere Mal Sprintduelle gegen den wie sediert wirkenden Mats Hummels. Toni Kroos hatte in der 21. Minute einen kapitalen Aussetzer und legte Gonzalo Higuaín den Ball mit einem Kopfball aus der Mitte der eigenen Hälfte zurück zum Torwart direkt in den Lauf, doch der verdutzte Argentinier vermasselte zum deutschen Glück seine Riesenchance. Offenkundig hatte dieser Torwart Manuel Neuer mit seiner Spielfeldhälftenbeherrschung einen gewissen furchteinflößenden Effekt auf den Gegner wie der Scheinriese bei Jim Knopf und Lukas der Lokomotivführer, denn eigentlich will Neuer ja nur spielen.\n\nNeun Minuten später traf Higuaín dann ins Tor, nachdem Ezequiel Lavezzi den Ball scharf hereingespielt hatte, doch Schiedsrichter Nicola Rizzoli aus Italien pfiff zum zweiten deutschen Glück berechtigterweise Abseits.\n\nKaum schien die deutsche Mannschaft sich allmählich mehr Sicherheit zu erspielen, musste schließlich der junge Kramer in der 32. Minute wieder seinen Platz räumen und sich durch André Schürrle ersetzen lassen: Der Abwehrmann Ezequiel Garay hatte ihn einige Minuten zuvor mit einem unzimperlichen Bodycheck abgeräumt, und Kramer schied mit einem Verdacht auf Gehirnerschütterung aus. Vor der Pause kamen die Deutschen nur noch zu einer erstklassigen Torchance: Benedikt Höwedes köpfte einen Eckball von Toni Kroos in der 45. Minute an den Pfosten.\n\nAuch nach dem Seitenwechsel wirkten die Argentinier torgefährlicher und Messi kam aus halblinker Position aus 16 Metern frei zum Torschuss, dem es aber an Präzision mangelte. Und in der 57. Minute musste Neuer mal wieder Kopf und Kragen riskieren: als er mit der Faust auf der äußersten linken Kante des Strafraums gegen Higuaín klärte, rammte er den Argentinier unsanft.\n\nIn der Schlussviertelstunde dann schien sich die Anstrengung der Argentinier mit Verlängerung und Elfmeterschießen plus dem Tag weniger Pause seit dem Halbfinale niederzuschlagen: Auch weil Trainer Alejandro Sabella mit den Einwechslungen von Sergio Agüero und Rodrigo Palacio das Offensivspiel eher ungewohnt forcierte, kamen die Deutschen zu besseren Torchancen durch Benedikt Höwedes und Toni Kroos. Obwohl die Partie spannender wurde, blieben Tore in der regulären Spielzeit aus.\n\nIn der Verlängerung starteten dann die Deutschen besser. Gleich nach zwei Minuten kam Schürrle von Halblinks frei zum Schuss, scheiterte aber am erneut sicheren Torwart Sergio Romero. Fünf Minuten später hatte der eingewechselte Palacio frei vor dem herausstürmenden Neuer bei seinem Heber die Führung auf dem Fuß, bugsierte den Ball aber statt ins nur neben das Tor. Längst war beiden Mannschaften deutlich die Strapaze und Ermüdung anzusehen. Die Verletzungen in den Zweikämpfen häuften sich: Allein Bastian Schweinsteiger musste sich zunächst nach einem Foul von Javier Mascherano am Knöchel treffen und kurz darauf von Agüero mit dem Ellbogen den Wangenknochen blutig schlagen lassen.\n\nDanach brachten zwei Einwechselspieler die Entscheidung: Schürrle spielte von der linken Seite den Ball in den Strafraum zu Mario Götze, der in der 88. Minute für Miroslav Klose gekommen war. Götze stoppte den Ball perfekt mit der Brust und schoss in der 113. Minute aus der Drehung ein − mit Links aus neun Metern, unhaltbar.\n\n# Beispiel 3\n\nNach einem dramatischen Finale gegen Argentinien feiert Deutschland den vierten WM-Titel. Der Treffer fiel erst in der Verlängerung. Lesen Sie das Spiel im Liveblog nach.\n\nEs war in der Verlängerung, als Bastian Schweinsteiger das gesamte WM-Finale plötzlich ins Gesicht geschrieben stand: Wiederholt gefoult von einem dauergrätschenden Javier Mascherano rappelte sich der Bayernspieler auf, schüttelte noch einen Krampf aus den Beinen und spielte weiter. Nur wenige Minuten später musste er mit einem Cut im Gesicht behandelt werden; als er zurückkam, sah er, wie Mario Götze mit einem Schuss in der 113. Minute das 1:0 erzielte und Deutschland zum vierten Weltmeistertitel schoss.\n\nDie Wiederauflage des Finales von 1990 lässt sich mit diesen Szenen zusammenfassen: Es war ruppig, bisweilen hart – und dramatisch bis zum Schluss. Bereits beim Aufwärmen verletzte sich Sami Khedira, der für ihn spielende Christoph Kramer musste bereits nach 32 Minuten nach einem rüden Bodycheck vom Platz. Überhaupt lief vieles gegen die deutsche Elf in der ersten Halbzeit: Messi und Lavezzi wirbelten, Higuaín erzielte ein Abseitstor, Kroos scheiterte an Romero, Höwedes traf kurz vor der Pause nur den Pfosten.\n\nDaran änderte sich auch nach dem Wechsel nichts. Die belächelten Argentinier, sie waren ebenbürtig, stellenweise sogar besser. Manuel Neuer, der zum Schluss den Goldenen Handschuh für den besten WM-Torhüter bekam, musste mehrfach eingreifen und die noch gegen Brasilien so einfachen Kombinationen prallten an der argentinischen Defensive ab. Erst gegen Ende kam die DFB-Elf wieder besser ins Spiel.\n\nViel wurde über die mangelnde spielerische Qualität dieser WM geschrieben. Dass es im Finale in die Verlängerung ging, passte in dieses Bild – und auch wieder nicht: Es gab zwar wenige längere Kombinationen, aber dafür umso mehr Dramatik. Schürrle traf kurz nach Wiederbeginn nur Torwart Romero, dann vertändelte Palacio den Ball. Und als die angeschlagenen Hummels und Schweinsteiger sich scheinbar nur noch mit Mühe auf den Beinen hielten, erzielte der zuvor eingewechselte Götze den entscheidenden Treffer. Der Wechselgott Joachim Löw? Auch diese Geschichte schrieb das Finale.\nApropos Geschichte: Zu Beginn dieses Spiels erinnerten wir uns an 1990, den letzten Titel. Andi Brehmes Strafstoß darf nun endgültig ruhen. Deutschland ist zum vierten Mal Weltmeister und wir fragen in spätestens vier Jahren: Wo waren Sie eigentlich am 13. Juli 2014?'
gpt4o_texts     = []
gpt35t_texts    = []
perplxty_texts  = []
clde_texts      = []

In [83]:
if int(TEST) == 1:   
    with open('imports/TEST-1', 'r') as f:
        human_texts = f.read()
else:
    with open(HUMAN_FILE, 'r') as f:
        human_texts = f.read()

human_texts = human_texts.split('--------------------NEW-TEXT------------------')

for i in human_texts:
    human_txt_list = i.split()
    prompt.append(f'Schreibe einen Fußballartikel zum Thema WM Sieg der deutschen National Mannschaft am 13.07.2014! Achten Sie auf fachliche korrektness und auf die im Systemprompt angegebenen Schemata! Der Text beginnt mit: {human_txt_list[0], human_txt_list[1], human_txt_list[2]}. Folgen Sie diesem Beginn! Der Artikel soll mindestens 700 und maximal 800 Wörter lang sein!')
    

In [84]:
l = []
for i in human_texts:
    human_txt_list = i.split()
    l.append(f'{human_txt_list[0]}, {human_txt_list[1]}, {human_txt_list[2]}')
    

In [85]:
print(l)

['So, reden, große', 'Die, deutsche, Fußball-Nationalelf', 'Es, ist, vollbracht:', 'Deutschland, ist, Weltmeister!', 'Drei, Minuten, vor', 'Da, ist, der', 'Die, DFB-Elf, gewinnt', 'Weltmeister!, Deutschland, ist', 'Die, DFB-Elf, hat', 'Der, eingewechselte, Mario', 'Geschafft!, Die, Deutsche', 'Rio, de, Janeiro', 'Deutschland, hatte, zunächst', 'Am, 13., Juli', 'Mit, einem, mühevollen', 'Nach, einem, dramatischen', 'Weltmeister!, Zum, ersten', '"Schlaaaaand!!!", Hunderttausende, Fußballfans', 'Obschon, der, lange', 'Dieses, Spiel!, Diese']


In [66]:
counter = 0
Path('backups').mkdir(parents=True, exist_ok=True)
for i in prompt:
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    
    perplxty_texts.append(perplxty_request(i, system_prompt))
    open(f'backups/perplexity_{timestamp}_{counter}.txt', 'w').write(perplxty_texts[- 1])
    print(f'Perplexity: {counter}')

    counter += 1

Perplexity: 0
Perplexity: 1
Perplexity: 2
Perplexity: 3
Perplexity: 4
Perplexity: 5
Perplexity: 6
Perplexity: 7
Perplexity: 8
Perplexity: 9
Perplexity: 10
Perplexity: 11
Perplexity: 12
Perplexity: 13
Perplexity: 14
Perplexity: 15
Perplexity: 16
Perplexity: 17
Perplexity: 18
Perplexity: 19


In [87]:
counter = 0
Path('imports').mkdir(parents=True, exist_ok=True)
Path('backups').mkdir(parents=True, exist_ok=True)

for i in prompt:
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    gpt4o_texts.append(gpt4o_request(i, system_prompt))
    print(f'GPT-4o: {counter}')
    gpt35t_texts.append(gpt35t_request(i, system_prompt))
    print(f'GPT-3.5-turbo: {counter}')
    perplxty_texts.append(perplxty_request(i, system_prompt))
    print(f'Perplexity: {counter}')
    clde_texts.append(clde_request(i, system_prompt))
    print(f'CLDE: {counter}')
    
    counter += 1

GPT-4o: 0
GPT-3.5-turbo: 0
Perplexity: 0
CLDE: 0
GPT-4o: 1
GPT-3.5-turbo: 1
Perplexity: 1
CLDE: 1
GPT-4o: 2
GPT-3.5-turbo: 2
Perplexity: 2
CLDE: 2
GPT-4o: 3
GPT-3.5-turbo: 3
Perplexity: 3
CLDE: 3
GPT-4o: 4
GPT-3.5-turbo: 4
Perplexity: 4
CLDE: 4
GPT-4o: 5
GPT-3.5-turbo: 5
Perplexity: 5
CLDE: 5
GPT-4o: 6
GPT-3.5-turbo: 6
Perplexity: 6
CLDE: 6
GPT-4o: 7
GPT-3.5-turbo: 7
Perplexity: 7
CLDE: 7
GPT-4o: 8
GPT-3.5-turbo: 8
Perplexity: 8
CLDE: 8
GPT-4o: 9
GPT-3.5-turbo: 9
Perplexity: 9
CLDE: 9
GPT-4o: 10
GPT-3.5-turbo: 10
Perplexity: 10
CLDE: 10
GPT-4o: 11
GPT-3.5-turbo: 11
Perplexity: 11
CLDE: 11
GPT-4o: 12
GPT-3.5-turbo: 12
Perplexity: 12
CLDE: 12
GPT-4o: 13
GPT-3.5-turbo: 13
Perplexity: 13
CLDE: 13
GPT-4o: 14
GPT-3.5-turbo: 14
Perplexity: 14
CLDE: 14
GPT-4o: 15
GPT-3.5-turbo: 15
Perplexity: 15
CLDE: 15
GPT-4o: 16
GPT-3.5-turbo: 16
Perplexity: 16
CLDE: 16
GPT-4o: 17
GPT-3.5-turbo: 17
Perplexity: 17
CLDE: 17
GPT-4o: 18
GPT-3.5-turbo: 18
Perplexity: 18
CLDE: 18
GPT-4o: 19
GPT-3.5-turbo: 19
Perp

In [88]:
for idx, i in enumerate(perplxty_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    perplxty_texts[idx] = new_text

In [89]:
for idx, i in enumerate(clde_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    clde_texts[idx] = new_text

In [90]:
for idx, i in enumerate(gpt4o_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    gpt4o_texts[idx] = new_text

In [91]:
for idx, i in enumerate(gpt35t_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    gpt35t_texts[idx] = new_text

In [92]:
marker = '\n--------------------NEW-TEXT------------------\n'

perplxty_texts_str = marker.join(perplxty_texts)
gpt4o_texts_str = marker.join(gpt4o_texts)
gpt35t_texts_str = marker.join(gpt35t_texts)
clde_texts_str = marker.join(clde_texts)
human_texts_str = marker.join(human_texts)

with open('exports/perplexity_full.txt', 'w') as f:
    f.write(perplxty_texts_str)

with open('exports/gpt4o_full.txt', 'w') as f:
    f.write(gpt4o_texts_str)

with open('exports/gpt35t_full.txt', 'w') as f:
    f.write(gpt35t_texts_str)

with open('exports/clde_full.txt', 'w') as f:
    f.write(clde_texts_str)

with open('exports/human_full.txt', 'w') as f:
    f.write(human_texts_str)

In [93]:
print(gpt4o_texts)
print(gpt35t_texts)
print(perplxty_texts)

['So reden große Erfolge – unvergesslich und tief in der Erinnerung verankert. Am 13. Juli 2014 schrieb die deutsche Fußballnationalmannschaft Geschichte, als sie im legendären Maracanã-Stadion von Rio de Janeiro den vierten Weltmeistertitel für Deutschland errang. Mit einem dramatischen 1:0-Sieg nach Verlängerung gegen ein hartnäckiges argentinisches Team sicherte sich die Mannschaft von Bundestrainer Joachim Löw einen Platz in den Annalen des Weltfußballs.\n\n Der Weg zum Titel\n\nDas Turnier in Brasilien begann für die deutsche Mannschaft vielversprechend. Bereits in der Gruppenphase setzte sie mit einem beeindruckenden 4:0 gegen Portugal ein klares Zeichen. Diesem folgten ein hart erkämpftes 2:2 gegen Ghana und ein solider 1:0-Erfolg über die USA. Deutschlands Spielweise verkörperte eine perfekte Mischung aus taktischer Disziplin, technischer Finesse und schier unermüdlichem Kampfgeist.\n\nBesonders denkwürdig war auch der 7:1-Triumph über Gastgeber Brasilien im Halbfinale – ein Sp

In [94]:
gpt4o_df    = pd.DataFrame({"words": []})
gpt35t_df   = pd.DataFrame({"words": []})
gemni_df    = pd.DataFrame({"words": []})
perplxty_df = pd.DataFrame({"words": []})
clde_df     = pd.DataFrame({"words": []})
human_df    = pd.DataFrame({"words": []})

In [95]:
nlp = spacy.load('de_core_news_sm')

In [96]:
for txt in gpt4o_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    gpt4o_df = pd.concat([gpt4o_df,txt_df], ignore_index=True)


In [97]:

for txt in gpt35t_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    gpt35t_df = pd.concat([gpt35t_df,txt_df], ignore_index=True)


In [98]:

for txt in perplxty_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    perplxty_df = pd.concat([perplxty_df,txt_df], ignore_index=True)


In [99]:

for txt in clde_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    clde_df = pd.concat([clde_df,txt_df], ignore_index=True)


In [100]:

for txt in human_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    human_df = pd.concat([human_df,txt_df], ignore_index=True)

In [101]:
gpt4o_df    = gpt4o_df.value_counts().reset_index()
gpt35t_df   = gpt35t_df.value_counts().reset_index()
perplxty_df = perplxty_df.value_counts().reset_index()
clde_df     = clde_df.value_counts().reset_index()
human_df    = human_df.value_counts().reset_index()

gpt4o_df.columns    = ['words', 'amount']
gpt35t_df.columns   = ['words', 'amount']
perplxty_df.columns = ['words', 'amount']
clde_df.columns     = ['words', 'amount']
human_df.columns    = ['words', 'amount']

In [102]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
Path('exports').mkdir(parents=True, exist_ok=True)

gpt4o_df.to_csv(f'exports/GPT4o-{timestamp}.csv', index=True)
gpt35t_df.to_csv(f'exports/GPT35t-{timestamp}.csv', index=True)
perplxty_df.to_csv(f'exports/PERPLEXITY-{timestamp}.csv', index=True)
clde_df.to_csv(f'exports/CLAUDE-{timestamp}.csv', index=True)
human_df.to_csv(f'exports/HUMAN-{timestamp}.csv', index=True)